#Classificazione con rappresentazione Doc2Vec

Compito di classificazione del testo rispetto alle seguenti categorie:


*   Automotive
*   Patio, Lawn and Garden
*   Office Products
*   Toys_&_Games



Mount Drive

In [ ]:
import random
random.seed(2020)

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


Import librerie

In [ ]:
!pip install contractions

     |████████████████████████████████| 317kB 4.6MB/s 
     |████████████████████████████████| 245kB 5.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81702 sha256=973734b70158aa9507d3b14754b491100f53eeb64c4a1d0bb72ccf5a9d923351
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
!pip install pyspellchecker

     |████████████████████████████████| 1.9MB 4.8MB/s 


In [ ]:
import gzip
import numpy as np
import json
import pandas as pd
from sklearn import utils
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords, treebank
import contractions
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()
from spellchecker import SpellChecker
from itertools import chain
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import seaborn as sns
import gensim
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pip install ipython-autotime
!pip install ipdb
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1831 sha256=b871e80c5bbbd625c6c43936c321b4a137de972d1fd053ff8210c881005a67cc
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime
  Created wheel for ipdb: filename=ipdb-0.13.3-cp36-none-any.whl size=10847 sha256=54f36ea924727151d1f3e7e440a5917d159287f0ca3f20ab1c1e673c4ec6ec4e
  Stored in directory: /root/.cache/pip/wheels/75/00/30/4169bcc3643f0cf946dcf37af1b71364b390c4df91da02b03c
Successfully built ipdb


In [ ]:
pd.set_option('display.max_colwidth', -1)

time: 1.81 ms


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


##Caricamento dati

In [ ]:
collection_stem = pd.read_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/post_stopwords_stem_filtro.csv")
collection_lemma = pd.read_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/post_stopwords_lemma_filtro.csv")

time: 19.3 s


In [ ]:
print(len(collection_stem), len(collection_lemma))

111860 111860
time: 1.16 ms


In [ ]:
collection_stem.drop(columns=["Unnamed: 0"], inplace = True)
collection_lemma.drop(columns=["Unnamed: 0"], inplace = True)

time: 76.9 ms


In [ ]:
collection_lemma = collection_lemma.sample(frac = 1, random_state = 2020)
collection_stem = collection_stem.sample(frac = 1, random_state = 2020)

time: 122 ms


In [ ]:
def tostring(tokens):
  tokens = str(tokens)
  tokens = tokens.replace("[", "")
  tokens = tokens.replace("]", "")
  tokens = tokens.replace("'", "")
  tokens = tokens.replace(",", "")
  return tokens

time: 3.4 ms


##Funzione di CV

In [ ]:
!pip install XGBoost

time: 2.16 s


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score

def Cross_Validation(features, target, classificatore):
  kfold = StratifiedKFold(n_splits=5, random_state=2020, shuffle=True)
  cvscores = []

  i = 1
  for train, test in kfold.split(features, target):
    X_train, X_val = features[train], features[test]
    y_train, y_val = target.values[list(train)], target.values[list(test)]

    if str(type(classificatore)) == "<class 'keras.engine.sequential.Sequential'>":
      label_encoder = LabelEncoder()
      y_train = label_encoder.fit_transform(y_train)
      y_val = label_encoder.fit_transform(y_val)
      # fit model
      classificatore.fit(X_train, y_train)
      # predict
      pred = classificatore.predict_class(X_val)
    else:
      # fit model
      classificatore.fit(X_train, y_train)
      # predict
      pred = classificatore.predict(X_val)
    
    accuracy = accuracy_score(y_val, pred)
    print('Iterazione numero: ' + str(i))
    print('Accuracy attuale del modello di classificazione: ' + str(accuracy))

    cvscores.append(accuracy)
    i +=1

  print("Il punteggio medio ottenuto dalla cross-validation è " + str(np.mean(cvscores)) + " con una deviazione di " + str(np.std(cvscores)))
  return cvscores


time: 28.8 ms


Dataframe per risultati

In [ ]:
results = pd.DataFrame(columns=["Algoritmo", "Rappresentazione", "Punteggio_cv"])
results

,Algoritmo,Rappresentazione,Punteggio_cv


time: 21.1 ms


#Creazione Doc2Vec e Classificazione

Per via dell'utilizzo intensivo delle risorse computazionali e di tempo messe a disposizione da Google Colab, si procede ad effettuare una rappresentazione Doc2Vec esclusivamente a partire da un vocabolario dati filtrato da parole troppo o troppo poco frequenti.

Tag dei documenti con la struttura richiesta dal modello Doc2Vec

In [ ]:
doctag_lemma = collection_lemma.apply(lambda doc: TaggedDocument(words=(doc.lemmanosw2), tags=[doc.target]), axis=1)
doctag_stem = collection_stem.apply(lambda doc: TaggedDocument(words=(doc.stemnosw2), tags=[doc.target]), axis=1)

time: 7.21 s


Creazione del vocabolario Doc2Vec, il modello scelto è dbow (simile a skipgram per word2vec che prevede il contesto attorno alla parola)

In [ ]:
dbow_lemma = Doc2Vec(dm=0, vector_size=300, hs=0, min_count=4, negative = 20, sample = 0)
dbow_lemma.build_vocab([x for x in tqdm(doctag_lemma.values)])

100%|██████████| 111860/111860 [00:00<00:00, 3460323.23it/s]


time: 5.16 s


In [ ]:
dbow_stem = Doc2Vec(dm=0, vector_size=300, hs=0, min_count=4, negative = 20, sample = 0)
dbow_stem.build_vocab([x for x in tqdm(doctag_stem.values)])

100%|██████████| 111860/111860 [00:00<00:00, 3195404.46it/s]


time: 5.02 s


Train del modello Doc2Vec

In [ ]:
#lemma
for epoch in range(10):
    dbow_lemma.train(utils.shuffle([x for x in tqdm(doctag_lemma.values)]), total_examples=len(doctag_lemma.values), epochs=1)
    dbow_lemma.alpha -= 0.002
    dbow_lemma.min_alpha = dbow_lemma.alpha

100%|██████████| 111860/111860 [00:00<00:00, 3627257.26it/s]


time: 26min 42s


In [ ]:
#stem
for epoch in range(10):
    dbow_stem.train(utils.shuffle([x for x in tqdm(doctag_stem.values)]), total_examples=len(doctag_stem.values), epochs=1)
    dbow_stem.alpha -= 0.002
    dbow_stem.min_alpha = dbow_stem.alpha

Creazione del vettore-rappresentazione finale per i classificatori:

In [ ]:
def crea_rappresentazione(modw2v, doctag):
    documenti = doctag.values
    target, features = zip(*[(doc.tags[0], modw2v.infer_vector(doc.words, steps=20)) for doc in documenti])
    return target, features

time: 1.8 ms


##Lemma Filter

In [ ]:
target_lemma, features_lemma = crea_rappresentazione(dbow_lemma, doctag_lemma)

time: 50min 53s


In [ ]:
write = open("/gdrive/My Drive/Progetto_Text_Mining/Collection/doc2vec_target_lemma", "wb")
pickle.dump(target_lemma, write)
write.close()

write2 = open("/gdrive/My Drive/Progetto_Text_Mining/Collection/doc2vec_features_lemma", "wb")
pickle.dump(features_lemma, write2)
write2.close()

time: 3.22 s


In [ ]:
results = pd.DataFrame(columns=["Algoritmo", "Rappresentazione", "Punteggio_cv"])

time: 6.83 ms


In [ ]:
features_lemma = np.asarray(features_lemma)
target_lemma = collection_lemma.target

time: 164 ms


In [ ]:
from sklearn.ensemble import RandomForestClassifier

#creazione del modello per CV
rf = RandomForestClassifier(n_estimators=100)

#richiamo funzione per CV
res = dict({"Algoritmo": "RandomForest", "Rappresentazione": "Doc2Vec", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, rf)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.5123368496334704
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.5164938315751832
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.5143035937779367
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.5140354013946004
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.5131861255140354
Il punteggio medio ottenuto dalla cross-validation è 0.5140711603790452 con una deviazione di 0.0013939698841068937


,Algoritmo,Rappresentazione,Punteggio_cv
0,RandomForest,Doc2Vec,"[0.5123368496334704, 0.5164938315751832, 0.5143035937779367, 0.5140354013946004, 0.5131861255140354]"


time: 16min 11s


In [ ]:
from sklearn import svm
SVM = svm.LinearSVC()
res = dict({"Algoritmo": "SVMLinear", "Rappresentazione": "Doc2Vec", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, SVM)})
results.append(res, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.4986590380833184


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.5028160200250313


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.49767566601108526
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.49620060790273557
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.4991954228499911
Il punteggio medio ottenuto dalla cross-validation è 0.49890935097443234 con una deviazione di 0.0022029899840719


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,Algoritmo,Rappresentazione,Punteggio_cv
0,SVMLinear,Doc2Vec,"[0.4986590380833184, 0.5028160200250313, 0.49767566601108526, 0.49620060790273557, 0.4991954228499911]"


time: 34min 16s


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 120)
res = dict({"Algoritmo": "KNN", "Rappresentazione": "Doc2Vec", "Punteggio_cv": Cross_Validation(features_lemma, target_lemma, KNN)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.4986143393527624
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.5046486679778295
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.49995530126944393
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.5021455390666905
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.5003128911138923
Il punteggio medio ottenuto dalla cross-validation è 0.5011353477561237 con una deviazione di 0.0020875025017031926


,Algoritmo,Rappresentazione,Punteggio_cv
0,KNN,Doc2Vec,"[0.4986143393527624, 0.5046486679778295, 0.49995530126944393, 0.5021455390666905, 0.5003128911138923]"


time: 1h 45min 31s


In [ ]:
results.to_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/risultati_Doc2Vec_lemma_filtro.csv")

time: 254 ms


##Stem Filter

In [ ]:
target_stem, features_stem = crea_rappresentazione(dbow_stem, doctag_stem)

time: 56min 30s


In [ ]:
write = open("/gdrive/My Drive/Progetto_Text_Mining/Collection/doc2vec_target_stem", "wb")
pickle.dump(target_stem, write)
write.close()

write2 = open("/gdrive/My Drive/Progetto_Text_Mining/Collection/doc2vec_features_stem", "wb")
pickle.dump(features_stem, write2)
write2.close()

time: 2.23 s


In [ ]:
results = pd.DataFrame(columns=["Algoritmo", "Rappresentazione", "Punteggio_cv"])

time: 25.2 ms


In [ ]:
features_stem = np.asarray(features_stem)
target_stem = collection_stem.target

time: 192 ms


In [ ]:
from sklearn.ensemble import RandomForestClassifier

#creazione del modello per CV
rf = RandomForestClassifier(n_estimators=100)

#richiamo funzione per CV
res = dict({"Algoritmo": "RandomForest", "Rappresentazione": "Doc2Vec", "Punteggio_cv": Cross_Validation(features_stem, target_stem, rf)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.5072411943500804
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.5096102270695512
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.5109511889862328
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.506034328625067
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.5076434829250849
Il punteggio medio ottenuto dalla cross-validation è 0.5082960843912032 con una deviazione di 0.001756745411702731


,Algoritmo,Rappresentazione,Punteggio_cv
0,RandomForest,Doc2Vec,"[0.5072411943500804, 0.5096102270695512, 0.5109511889862328, 0.506034328625067, 0.5076434829250849]"


time: 17min 43s


In [ ]:
from sklearn import svm
SVM = svm.LinearSVC()
res = dict({"Algoritmo": "SVMLinear", "Rappresentazione": "Doc2Vec", "Punteggio_cv": Cross_Validation(features_stem, target_stem, SVM)})
results.append(res, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.4964241015555158


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.5021455390666905


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


...

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.4941891650277132
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.4937421777221527
Il punteggio medio ottenuto dalla cross-validation è 0.4969604863221885 con una deviazione di 0.0030662156286032057


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,Algoritmo,Rappresentazione,Punteggio_cv
0,SVMLinear,Doc2Vec,"[0.4964241015555158, 0.5021455390666905, 0.49830144823887, 0.4941891650277132, 0.4937421777221527]"


time: 50min 50s


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 120)
res = dict({"Algoritmo": "KNN", "Rappresentazione": "Doc2Vec", "Punteggio_cv": Cross_Validation(features_stem, target_stem, KNN)})
results.append(res, ignore_index=True)

Iterazione numero: 1
Accuracy attuale del modello di classificazione: 0.5013856606472377
Iterazione numero: 2
Accuracy attuale del modello di classificazione: 0.5084033613445378
Iterazione numero: 3
Accuracy attuale del modello di classificazione: 0.5012515644555695
Iterazione numero: 4
Accuracy attuale del modello di classificazione: 0.5025031289111389
Iterazione numero: 5
Accuracy attuale del modello di classificazione: 0.5024584301805829
Il punteggio medio ottenuto dalla cross-validation è 0.5032004291078134 con una deviazione di 0.0026532527897094715


,Algoritmo,Rappresentazione,Punteggio_cv
0,KNN,Doc2Vec,"[0.5013856606472377, 0.5084033613445378, 0.5012515644555695, 0.5025031289111389, 0.5024584301805829]"


time: 2h 12min 8s


In [ ]:
results.to_csv("/gdrive/My Drive/Progetto_Text_Mining/Collection/risultati_Doc2Vec_stem_filtro.csv")

time: 699 ms
